# FINAL PROJECT: ÍÑIGO BARCELÓ ÁLVAREZ / JUAN LEAL ALIAGA

### The document is divided in sections, for each exercise asked to perform, there is a section with the implementation and another one with the tests conducted to ensure the good performance of the implementation

## 1. Types to represent all the former concepts. An exhaustive type definition is required, use the most appropriate way (type, data or newtype) for each of them. 

In [25]:
-- First datatype is Date which includes a day (integer), a month (string) and a year (integer). We derive from show as we will print it, 
--from eq and ord as we will be comparing dates to perform actions
data Date = Date 
  { day   :: Int
  , month :: String
  , year  :: Int
  } deriving (Show, Eq, Ord)
--Second datatype is person having firstname and lastname, both strings and a birthDate which is a date we also derive from show to print and
-- and eq to compare with other people
data Person = Person 
  { firstName :: String
  , lastName  :: String
  , birthDate :: Date
  } deriving (Show, Eq)
-- Publication can be one of three possibilities (Book, Journal, DVD) each of one having its own constructor, for the copies functionality we
-- used a vector that would keep track of the copies (type defined below) of the publication itself
data Publication
  = Book
      { bookId    :: String       -- Unique signature
      , author    :: Person       -- Single author
      , title     :: String
      , pages     :: Int
      , copies    :: [Copy]       -- List of copies
      }
  | Journal
      { journalId :: String       -- Unique signature
      , authors   :: [Person]     -- Multiple authors
      , title     :: String
      , releaseDate :: Date
      , pages     :: Int
      , copies    :: [Copy]
      }
  | DVD
      { dvdId     :: String       -- Unique signature
      , director  :: Person       -- Director is a single person
      , actors    :: [Person]     -- List of actors
      , title     :: String
      , releaseDate :: Date
      , duration  :: Int          -- Duration in minutes
      , copies    :: [Copy]
      }
  deriving (Show, Eq)
-- Copy has one mandatory parameter which is a bool to know if its borrowed and in case it is borrowed, it has three other parameters to keep
-- track 
data Copy = Copy
  { isBorrowed  :: Bool
  , borrower    :: Maybe User    -- User who borrowed the copy (if any)
  , loanDate    :: Maybe Date    -- Loan date (if borrowed)
  , returnDate  :: Maybe Date    -- Expected return date (if borrowed)
  } deriving (Show, Eq)

-- User hsd two constructors, its either a student or a professor which are both a person
data User
  = Student 
      { studentName :: Person
      }
  | Professor
      { professorName :: Person
      }
  
  deriving (Show, Eq)
-- Catalog is a list of publications
type Catalog = [Publication]

## 2. Functions to show the former concepts in a pretty way. 

In [26]:

--Redefining the show instance for every type so it prints in a pretty way 
instance Show Date where
  show (Date d m y) = show d ++ " " ++ m ++ " " ++ show y

instance Show Person where
  show (Person fName lName bDate) = 
    fName ++ " " ++ lName ++ " (" ++ show bDate ++ ")"
    
instance Show Copy where
  show (Copy isB mbUser mbLoanDate mbReturnDate)
    | isB = "Borrowed by " ++ maybe "Unknown" show mbUser ++
            " | Loan Date: " ++ maybe "N/A" show mbLoanDate ++
            " | Return Date: " ++ maybe "N/A" show mbReturnDate
    | otherwise = "Available"
    
instance Show Publication where
  show (Book id author title pages copies) =
    "Book [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Author: " ++ show author ++ "\n" ++
    "  Pages: " ++ show pages ++ "\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)

  show (Journal id authors title releaseDate pages copies) =
    "Journal [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Authors: " ++ unwords (map show authors) ++ "\n" ++
    "  Release Date: " ++ show releaseDate ++ "\n" ++
    "  Pages: " ++ show pages ++ "\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)

  show (DVD id director actors title releaseDate duration copies) =
    "DVD [" ++ id ++ "]\n" ++
    "  Title: " ++ title ++ "\n" ++
    "  Director: " ++ show director ++ "\n" ++
    "  Actors: " ++ unwords (map show actors) ++ "\n" ++
    "  Release Date: " ++ show releaseDate ++ "\n" ++
    "  Duration: " ++ show duration ++ " minutes\n" ++
    "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
    unlines (map show copies)
    
instance Show User where
    show (Student p) = 
        "Student: " ++ show p ++ "\n"
    
    show (Professor p) = 
        "Professor: " ++ show p ++ "\n"
    
instance Eq User where
    (Student p1) == (Student p2) = p1 == p2
    (Professor p1) == (Professor p2) = p1 == p2
    _ == _ = False
    

prettyCatalog :: Catalog -> IO ()
prettyCatalog [] = putStrLn "[]"
prettyCatalog catalog = do
  putStrLn "["
  mapM_ (\pub -> do
           putStrLn $ prettyPublication pub ++ ","
         ) (init catalog)
  putStrLn $ prettyPublication (last catalog)
  putStrLn "]"
  where
    -- Pretty print a single publication
    prettyPublication :: Publication -> String
    prettyPublication (Book id author title pages copies) =
      "Book [" ++ id ++ "]\n" ++
      "  Title: " ++ title ++ "\n" ++
      "  Author: " ++ show author ++ "\n" ++
      "  Pages: " ++ show pages ++ "\n" ++
      "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
      concatMap (\c -> "  " ++ show c ++ "\n") copies

    prettyPublication (Journal id authors title releaseDate pages copies) =
      "Journal [" ++ id ++ "]\n" ++
      "  Title: " ++ title ++ "\n" ++
      "  Authors: " ++ unwords (map show authors) ++ "\n" ++
      "  Release Date: " ++ show releaseDate ++ "\n" ++
      "  Pages: " ++ show pages ++ "\n" ++
      "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
      concatMap (\c -> "  " ++ show c ++ "\n") copies

    prettyPublication (DVD id director actors title releaseDate duration copies) =
      "DVD [" ++ id ++ "]\n" ++
      "  Title: " ++ title ++ "\n" ++
      "  Director: " ++ show director ++ "\n" ++
      "  Actors: " ++ unwords (map show actors) ++ "\n" ++
      "  Release Date: " ++ show releaseDate ++ "\n" ++
      "  Duration: " ++ show duration ++ " minutes\n" ++
      "  Copies: " ++ show (length $ filter (not . isBorrowed) copies) ++ "/" ++ show (length copies) ++ " available\n" ++
      concatMap (\c -> "  " ++ show c ++ "\n") copies
      

In [27]:
exampleBook :: Publication
exampleBook = Book 
  "B001" 
  (Person "Jane" "Austen" (Date 16 "December" 1775)) 
  "Functional Programming in Haskell" 
  300 
  [Copy False Nothing Nothing Nothing]
newPrideAndPrejudice :: Publication
newPrideAndPrejudice = Book
  "B002"
  (Person "Jane" "Austen" (Date 16 "December" 1775))
  "Pride and Prejudice"
  432
  [Copy False Nothing Nothing Nothing]

exampleCatalog :: Catalog
exampleCatalog = [exampleBook, newPrideAndPrejudice]
exampleCatalog
"---------"
prettyCatalog exampleCatalog

[Book [B001]
  Title: Functional Programming in Haskell
  Author: Jane Austen (16 December 1775)
  Pages: 300
  Copies: 1/1 available
Available
,Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 1/1 available
Available
]

"---------"

[
Book [B001]
  Title: Functional Programming in Haskell
  Author: Jane Austen (16 December 1775)
  Pages: 300
  Copies: 1/1 available
  Available
,
Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 1/1 available
  Available

]

# Test print Date, author, book, student and catalog

In [28]:
--Prueba
exampleDate :: Date
exampleDate = Date 18 "November" 2024

author1 :: Person
author1 = Person "John" "Doe" (Date 10 "January" 1980)

exampleBook :: Publication
exampleBook = Book 
  "B001" 
  author1 
  "Functional Programming in Haskell" 
  300 
  [Copy False Nothing Nothing Nothing]

exampleStudent :: User
exampleStudent = Student (Person "Alice" "Smith" (Date 5 "May" 2001))

exampleCatalog :: Catalog
exampleCatalog = [exampleBook]

exampleDate

author1

exampleBook

exampleStudent

exampleCatalog


18 November 2024

John Doe (10 January 1980)

Book [B001]
  Title: Functional Programming in Haskell
  Author: John Doe (10 January 1980)
  Pages: 300
  Copies: 1/1 available
Available

Student: Alice Smith (5 May 2001)

[Book [B001]
  Title: Functional Programming in Haskell
  Author: John Doe (10 January 1980)
  Pages: 300
  Copies: 1/1 available
Available
]

# Test  copies in publications, student and professor and show of catalog

In [29]:
-- Date example
exampleLoanDate :: Date
exampleLoanDate = Date 10 "November" 2024

exampleReturnDate :: Date
exampleReturnDate = Date 10 "December" 2024

-- Person example
student1 :: User
student1 = Student (Person "Alice" "Smith" (Date 5 "May" 2001))
professor1 :: User
professor1 = Professor (Person "Dr." "Johnson" (Date 12 "August" 1975)) 
-- Book copies example
exampleCopies :: [Copy]
exampleCopies = 
  [ Copy True (Just student1) (Just exampleLoanDate) (Just exampleReturnDate)  -- to a student 
  , Copy True (Just professor1) (Just exampleLoanDate) (Just exampleReturnDate) -- to a professor
  , Copy False Nothing Nothing Nothing  -- Available
  , Copy False Nothing Nothing Nothing  -- Available
  ]

-- Book with copies
exampleBookWithCopies :: Publication
exampleBookWithCopies = Book "B002" (Person "Jane" "Austen" (Date 16 "December" 1775)) "Pride and Prejudice" 432 [Copy False Nothing Nothing Nothing]

-- Catalog with the book
exampleCatalogWithCopies :: Catalog
exampleCatalogWithCopies = [exampleBookWithCopies]

exampleBookWithCopies

exampleCatalogWithCopies


exampleCopies2 :: [Copy]
exampleCopies2 = 
  [ Copy True (Just student1) (Just (Date 1 "November" 2024)) (Just (Date 8 "November" 2024)) -- To a student 
  , Copy False Nothing Nothing Nothing  -- Available
  , Copy False Nothing Nothing Nothing  -- Available
  ]

exampleBookWithCopies2 :: Publication
exampleBookWithCopies2 = Book
  "B003"
  (Person "George" "Orwell" (Date 25 "June" 1903))
  "1984"
  328
  exampleCopies2

-- Updated catalog
updatedCatalog :: Catalog
updatedCatalog = exampleCatalogWithCopies ++ [exampleBookWithCopies2]

-- Printing the catalog
prettyCatalog updatedCatalog
-- Proof empty catalog
prettyCatalog []

Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 1/1 available
Available

[Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 1/1 available
Available
]

[
Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 1/1 available
  Available
,
Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
  Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024
  Available
  Available

]

[]

## 3. An addToCatalog Publication Catalog function that adds a publication to the library catalog. If the publication is already in the catalog, its number of copies will be increased. 

In [30]:
addToCatalog :: Publication -> Catalog -> Catalog
addToCatalog pub [] = [pub]
addToCatalog pub (p:ps)
  | samePublication pub p = mergeCopies pub p : ps -- If it already exists then combines the copies 
  | otherwise             = p : addToCatalog pub ps -- If not continues
  where
    -- Check if two publications are the same looking the ID 
    samePublication :: Publication -> Publication -> Bool
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    -- Function to combine copies in the case they are the same publication
    mergeCopies :: Publication -> Publication -> Publication
    mergeCopies (Book id1 author1 title1 pages1 copies1) (Book _ _ _ _ copies2) =
      Book id1 author1 title1 pages1 (copies1 ++ copies2)
    mergeCopies (Journal id1 authors1 title1 release1 pages1 copies1) (Journal _ _ _ _ _ copies2) =
      Journal id1 authors1 title1 release1 pages1 (copies1 ++ copies2)
    mergeCopies (DVD id1 director1 actors1 title1 release1 duration1 copies1) (DVD _ _ _ _ _ _ copies2) =
      DVD id1 director1 actors1 title1 release1 duration1 (copies1 ++ copies2)
    mergeCopies pub1 _ = pub1 -- Por seguridad, devuelve pub1 en otros casos

# Tests for AddToCatalog

In [31]:
-- New copies for an existing book
newCopiesForPrideAndPrejudice :: [Copy]
newCopiesForPrideAndPrejudice = 
  [ Copy False Nothing Nothing Nothing,  -- New copy available
    Copy False Nothing Nothing Nothing   -- Another new copy available
  ]

-- Publication with new copies of "Pride and Prejudice"
newPrideAndPrejudice :: Publication
newPrideAndPrejudice = Book
  "B002"
  (Person "Jane" "Austen" (Date 16 "December" 1775))
  "Pride and Prejudice"
  432
  newCopiesForPrideAndPrejudice

-- A completely new book
newBook :: Publication
newBook = Book
  "B004"
  (Person "J.K." "Rowling" (Date 31 "July" 1965))
  "Harry Potter and the Philosopher's Stone"
  223
  [Copy False Nothing Nothing Nothing]

-- Update the catalog
updatedCatalog2 :: Catalog
updatedCatalog2 = []
"Print Catalog initialized "
prettyCatalog updatedCatalog2
"Print Catalog after function AddToCatalog "
updatedCatalog2 = addToCatalog newPrideAndPrejudice $ addToCatalog newBook updatedCatalog

-- Print test
prettyCatalog updatedCatalog2
"-------------------------------------"
"Catalog updated with new copies from a book"
-- New copies for "Harry Potter and the Philosopher's Stone"
newCopiesForHarryPotter :: [Copy]
newCopiesForHarryPotter = 
  [ Copy False Nothing Nothing Nothing,  -- New copy available
    Copy False Nothing Nothing Nothing   -- Another new copy available
  ]

-- Publication with new copies of "Harry Potter"
newBook2 :: Publication
newBook2 = Book
  "B004"
  (Person "J.K." "Rowling" (Date 31 "July" 1965))
  "Harry Potter and the Philosopher's Stone"
  223
  newCopiesForHarryPotter

-- Update the catalog with newBook2 (merge copies)
finalCatalog :: Catalog
finalCatalog = addToCatalog newBook2 updatedCatalog2

-- Print test
prettyCatalog finalCatalog
"-----Empty Catalog--------"
addToCatalog newBook2 []


"Print Catalog initialized "

[]

"Print Catalog after function AddToCatalog "

[
Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 3/3 available
  Available
  Available
  Available
,
Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
  Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024
  Available
  Available
,
Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 1/1 available
  Available

]

"-------------------------------------"

"Catalog updated with new copies from a book"

[
Book [B002]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 3/3 available
  Available
  Available
  Available
,
Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 2/3 available
  Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024
  Available
  Available
,
Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 3/3 available
  Available
  Available
  Available

]

"-----Empty Catalog--------"

[Book [B004]
  Title: Harry Potter and the Philosopher's Stone
  Author: J.K. Rowling (31 July 1965)
  Pages: 223
  Copies: 2/2 available
Available
Available
]

## 4. A removeFromCatalog Publication Catalog that removes a copy of the publication from the library catalog. If it is the last copy, the publication will be removed. If the catalog does not contain that publication an error will be raised.

In [32]:

removeFromCatalog :: Publication -> Catalog -> Either String Catalog
removeFromCatalog pub [] = Left "Error: The publication is not in the catalog."
removeFromCatalog pub (p:ps)
  | samePublication pub p =
      let updatedPub = removeCopy pub
      in if noCopiesLeft updatedPub
         then Right ps -- EErase the publication
         else Right (updatedPub : ps) -- Updates the publication
  | otherwise = fmap (p :) (removeFromCatalog pub ps) -- Still looking
  where
    -- Check if two pubications are the same
    samePublication :: Publication -> Publication -> Bool
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    -- Erases a copyof the publication first available or borrowed
    removeCopy :: Publication -> Publication
    removeCopy (Book id1 author1 title1 pages1 copies1) =
      Book id1 author1 title1 pages1 (tail copies1)
    removeCopy (Journal id1 authors1 title1 release1 pages1 copies1) =
      Journal id1 authors1 title1 release1 pages1 (tail copies1)
    removeCopy (DVD id1 director1 actors1 title1 release1 duration1 copies1) =
      DVD id1 director1 actors1 title1 release1 duration1 (tail copies1)
    removeCopy pub = pub -- Por seguridad

    -- Check if there are no copies left
    noCopiesLeft :: Publication -> Bool
    noCopiesLeft (Book _ _ _ _ copies1)    = null copies1
    noCopiesLeft (Journal _ _ _ _ _ copies1) = null copies1
    noCopiesLeft (DVD _ _ _ _ _ _ copies1) = null copies1



{-
removeFromCatalog :: Publication -> Catalog -> Either String Catalog
removeFromCatalog pub [] = Left "Error: The publication is not in the catalog."
removeFromCatalog pub (p:ps)
  | samePublication pub p =
      let updatedPub = removeCopy pub
      in if updatedPub == p
         then Right (p : ps) -- Ninguna copia estaba disponible; devuelve el catálogo como estaba
         else if noCopiesLeft updatedPub
              then Right ps -- Elimina la publicación completamente
              else Right (updatedPub : ps) -- Actualiza la publicación con las copias restantes
  | otherwise = fmap (p :) (removeFromCatalog pub ps) -- Sigue buscando
  where
    -- Verifica si dos publicaciones son iguales por su ID único
    samePublication :: Publication -> Publication -> Bool
    samePublication (Book id1 _ _ _ _) (Book id2 _ _ _ _) = id1 == id2
    samePublication (Journal id1 _ _ _ _ _) (Journal id2 _ _ _ _ _) = id1 == id2
    samePublication (DVD id1 _ _ _ _ _ _) (DVD id2 _ _ _ _ _ _) = id1 == id2
    samePublication _ _ = False

    -- Intenta eliminar una copia disponible (no prestada)
    removeCopy :: Publication -> Publication
    removeCopy (Book id1 author1 title1 pages1 copies1) =
      case break (not . isBorrowed) copies1 of
        (_, []) -> Book id1 author1 title1 pages1 copies1 -- No hay copias disponibles
        (before, available:after) ->
          Book id1 author1 title1 pages1 (before ++ after) -- Elimina la primera copia disponible
    removeCopy (Journal id1 authors1 title1 release1 pages1 copies1) =
      case break (not . isBorrowed) copies1 of
        (_, []) -> Journal id1 authors1 title1 release1 pages1 copies1 -- No hay copias disponibles
        (before, available:after) ->
          Journal id1 authors1 title1 release1 pages1 (before ++ after) -- Elimina la primera copia disponible
    removeCopy (DVD id1 director1 actors1 title1 release1 duration1 copies1) =
      case break (not . isBorrowed) copies1 of
        (_, []) -> DVD id1 director1 actors1 title1 release1 duration1 copies1 -- No hay copias disponibles
        (before, available:after) ->
          DVD id1 director1 actors1 title1 release1 duration1 (before ++ after) -- Elimina la primera copia disponible
    removeCopy pub = pub -- Por seguridad

    -- Verifica si no quedan copias en la publicación
    noCopiesLeft :: Publication -> Bool
    noCopiesLeft (Book _ _ _ _ copies1)    = null copies1
    noCopiesLeft (Journal _ _ _ _ _ copies1) = null copies1
    noCopiesLeft (DVD _ _ _ _ _ _ copies1) = null copies1
-}


# Tests for removeFromCatalog

In [33]:
exampleCopies3 :: [Copy]
exampleCopies3 = 
  [ Copy True (Just student1) (Just (Date 1 "November" 2024)) (Just (Date 8 "November" 2024))
  ]
exampleBookWithCopies3 :: Publication
exampleBookWithCopies3 = Book
  "B003"
  (Person "George" "Orwell" (Date 25 "June" 1903))
  "1984"
  328
  exampleCopies3
--exampleBookWithCopies3 :: Publication
exampleCopies4 :: [Copy]
exampleCopies4 = 
  [ Copy False Nothing Nothing Nothing, Copy True (Just student1) (Just (Date 1 "November" 2024)) (Just (Date 8 "November" 2024))
  ]
exampleBookWithCopies4 :: Publication
exampleBookWithCopies4 = Book
  "B003"
  (Person "George" "Orwell" (Date 25 "June" 1903))
  "1984"
  328
  exampleCopies4
--catalogForRemoval4 :: Catalog
--catalogForRemoval3 :: Catalog
catalogForRemoval4= [exampleBookWithCopies4]
catalogForRemoval3 = [exampleBookWithCopies3]
-- Eliminar una copia de "Pride and Prejudice"
--result1 :: Either String Catalog
"----------------------------"
exampleBookWithCopies3
exampleBookWithCopies4
--prettyCatalog catalogForRemoval
"--------------------"
removeFromCatalog exampleBookWithCopies4 catalogForRemoval4
"------------------------"
removeFromCatalog exampleBookWithCopies3 catalogForRemoval3
"------------------------"
removeFromCatalog exampleBookWithCopies3 []


"----------------------------"

Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 0/1 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024

Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 1/2 available
Available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024

"--------------------"

Right [Book [B003]
  Title: 1984
  Author: George Orwell (25 June 1903)
  Pages: 328
  Copies: 0/1 available
Borrowed by Student: Alice Smith (5 May 2001)
 | Loan Date: 1 November 2024 | Return Date: 8 November 2024
]

"------------------------"

Right []

"------------------------"

Left "Error: The publication is not in the catalog."

## 5. A borrowedPublications User Catalog function that returns the publications the user has borrowed. They must be sort by closest return date, in case of tie in return date, it will first show books, then journals and finally DVDs. 

In [34]:
-- Main function
borrowedPublications :: User -> Catalog -> [Publication]
borrowedPublications user catalog = 
  let borrowedCopies = [ publication | publication <- catalog, 
                                     any (isBorrowedByUser user) (copies publication) ]
  in sortPublications borrowedCopies

-- Auxiliar function to check if a copy is already borrowed
isBorrowedByUser :: User -> Copy -> Bool
isBorrowedByUser user copy = 
  case borrower copy of
    Just u  -> u == user  -- Si el prestatario es el usuario, es una copia tomada por él.
    Nothing -> False      -- NOt borrowed

-- Fucntion to sort publications
sortPublications :: [Publication] -> [Publication]
sortPublications [] = []
sortPublications (x:xs) = insertPublication x (sortPublications xs)

-- Function to insert a pub in the right place
insertPublication :: Publication -> [Publication] -> [Publication]
insertPublication pub [] = [pub]
insertPublication pub (x:xs)
  | comparator pub x == LT = pub : x : xs  -- Si pub debe ir antes de x
  | otherwise = x : insertPublication pub xs  -- Sino, seguimos buscando

-- Fucntion to order the publications
comparator :: Publication -> Publication -> Ordering
comparator pub1 pub2 = 
  case compareReturnDate pub1 pub2 of
    EQ -> compareType pub1 pub2
    result -> result

-- Function to check return dates
compareReturnDate :: Publication -> Publication -> Ordering
compareReturnDate pub1 pub2 =
  let returnDate1 = minimum (map returnDate (copies pub1))
      returnDate2 = minimum (map returnDate (copies pub2))
  in compare returnDate1 returnDate2

-- First book, then journal and finally dvd
compareType :: Publication -> Publication -> Ordering
compareType (Book _ _ _ _ _) (Book _ _ _ _ _) = EQ
compareType (Book _ _ _ _ _) _ = LT
compareType _ (Book _ _ _ _ _) = GT
compareType (Journal _ _ _ _ _ _) (Journal _ _ _ _ _ _) = EQ
compareType (Journal _ _ _ _ _ _) _ = LT
compareType _ (Journal _ _ _ _ _ _) = GT
compareType (DVD _ _ _ _ _ _ _) (DVD _ _ _ _ _ _ _) = EQ
compareType (DVD _ _ _ _ _ _ _) _ = LT
compareType _ (DVD _ _ _ _ _ _ _) = GT

Line 17: Use foldr
Found:
sortPublications [] = []
sortPublications (x : xs)
  = insertPublication x (sortPublications xs)
Why not:
sortPublications xs = foldr insertPublication [] xsLine 43: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 43: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 44: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 45: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 46: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 46: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 47: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 48: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 49: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 49: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 50: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 51: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}

# Test for BorrowedPublications

In [35]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

testUser = Student 
  (Person "John" "Doe" (Date 1 "January" 2000)) 

--
borrowedCopy1 = Copy True (Just testUser) (Just (Date 20 "November" 2024)) (Just (Date 30 "November" 2024))
borrowedCopy2 = Copy True (Just testUser) (Just (Date 15 "November" 2024)) (Just (Date 30 "November" 2024))
borrowedCopy3 = Copy True (Just testUser) (Just (Date 18 "November" 2024)) (Just (Date 28 "November" 2024))
borrowedCopy4 = Copy True (Just testUser) (Just (Date 24 "November" 2024)) (Just (Date 28 "November" 2024))
borrowedCopy5 = Copy False Nothing Nothing Nothing
book1, book2, journal1 :: Publication
book1 = Book "B001" (Person "Author1" "Surname1" (Date 1 "January" 1970)) 
                   "Functional Programming in Haskell" 300 [borrowedCopy1]
book2 = Book "B002" (Person "Author2" "Surname2" (Date 1 "February" 1980)) 
                   "Learn You a Haskell" 250 [borrowedCopy2]
journal1 = Journal "J001" [Person "Author3" "Surname3" (Date 1 "March" 1990)] 
                         "Advanced Haskell Research" (Date 1 "March" 2024) 
                         50 [borrowedCopy3, borrowedCopy5]
--dvd1 = DVD "D001" testUser [testUser] "Literature on Screen" (Date 1 "April" 2024) 120 []
dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 [borrowedCopy4]

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]

In [36]:
testUser
print("-----------------------------------")
testCatalog
print("-----------------------------------")
borrowedPublications testUser testCatalog
"------------------------------"
--With empty catalog
borrowedPublications testUser []

Line 2: Redundant bracket
Found:
("-----------------------------------")
Why not:
"-----------------------------------"Line 4: Redundant bracket
Found:
("-----------------------------------")
Why not:
"-----------------------------------"

Student: John Doe (1 January 2000)

"-----------------------------------"

[Book [B001]
  Title: Functional Programming in Haskell
  Author: Author1 Surname1 (1 January 1970)
  Pages: 300
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 20 November 2024 | Return Date: 30 November 2024
,Book [B002]
  Title: Learn You a Haskell
  Author: Author2 Surname2 (1 February 1980)
  Pages: 250
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 15 November 2024 | Return Date: 30 November 2024
,Journal [J001]
  Title: Advanced Haskell Research
  Authors: Author3 Surname3 (1 March 1990)
  Release Date: 1 March 2024
  Pages: 50
  Copies: 1/2 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 18 November 2024 | Return Date: 28 November 2024
Available
,DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/1 available
Borrowed

"-----------------------------------"

[Journal [J001]
  Title: Advanced Haskell Research
  Authors: Author3 Surname3 (1 March 1990)
  Release Date: 1 March 2024
  Pages: 50
  Copies: 1/2 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 18 November 2024 | Return Date: 28 November 2024
Available
,DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 24 November 2024 | Return Date: 28 November 2024
,Book [B002]
  Title: Learn You a Haskell
  Author: Author2 Surname2 (1 February 1980)
  Pages: 250
  Copies: 0/1 available
Borrowed by Student: John Doe (1 January 2000)
 | Loan Date: 15 November 2024 | Return Date: 30 November 2024
,Book [B001]
  Title: Functional Programming in Haskell
  Author: Author1 Surname1 (1 January 1970)
  Pages: 300
  Copies: 0/1 available
Borrowed

"------------------------------"

[]

## 6. A publicationsByAuthor Author Catalog that returns a list with all the publications for that author in the Catalog or the empty list if that person has not authored any publication. Create also equivalent booksByAuthor, journalsByAuthor and DVDsByAuthor that restrict the search to books, journals and DVDs respectively. Notice in DVD both directors and actors must be considered. 

In [37]:
-- General function to find all publications by a given author
publicationsByAuthor :: Person -> Catalog -> [Publication]
publicationsByAuthor author catalog =
  filter authoredBy catalog
  where
    -- Checks if a publication is authored by the given person
    authoredBy :: Publication -> Bool
    authoredBy (Book _ bookAuthor _ _ _) = bookAuthor == author
    authoredBy (Journal _ journalAuthors _ _ _ _) = author `elem` journalAuthors
    authoredBy (DVD _ director actors _ _ _ _) = director == author || author `elem` actors
    
booksByAuthor :: Person -> Catalog -> [Publication]
booksByAuthor author catalog =
  filter authoredByBook catalog
  where
    -- Checks if a book is authored by the given person
    authoredByBook :: Publication -> Bool
    authoredByBook (Book _ bookAuthor _ _ _) = bookAuthor == author
    authoredByBook _ = False

journalsByAuthor :: Person -> Catalog -> [Publication]
journalsByAuthor author catalog =
  filter authoredByJournal catalog
  where
    -- Checks if a journal is authored by the given person
    authoredByJournal :: Publication -> Bool
    authoredByJournal (Journal _ journalAuthors _ _ _ _) = author `elem` journalAuthors
    authoredByJournal _ = False

dvdsByAuthor :: Person -> Catalog -> [Publication]
dvdsByAuthor author catalog =
  filter authoredByDVD catalog
  where
    -- Checks if a DVD is directed or acted by the given person
    authoredByDVD :: Publication -> Bool
    authoredByDVD (DVD _ director actors _ _ _ _) = director == author || author `elem` actors
    authoredByDVD _ = False


Line 3: Eta reduce
Found:
publicationsByAuthor author catalog = filter authoredBy catalog
Why not:
publicationsByAuthor author = filter authoredByLine 13: Eta reduce
Found:
booksByAuthor author catalog = filter authoredByBook catalog
Why not:
booksByAuthor author = filter authoredByBookLine 22: Eta reduce
Found:
journalsByAuthor author catalog = filter authoredByJournal catalog
Why not:
journalsByAuthor author = filter authoredByJournalLine 31: Eta reduce
Found:
dvdsByAuthor author catalog = filter authoredByDVD catalog
Why not:
dvdsByAuthor author = filter authoredByDVD

# Tests for publicationsByAuthor

In [38]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

book1 :: Publication
book1 = Book "B001" author1 "Pride and Prejudice" 432 []

book2 :: Publication
book2 = Book "B002" author2 "Oliver Twist" 300 []

journal1 :: Publication
journal1 = Journal "J001" [author1, author2] "Victorian Literature" (Date 1 "January" 2024) 100 []

dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 []

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]


In [39]:
publicationsByAuthor author1 testCatalog
"------------------------------------"
booksByAuthor author1 testCatalog
"------------------------------------"
journalsByAuthor author1 testCatalog
"------------------------------------"
dvdsByAuthor author2 testCatalog
"------------------------------------"
"With empty catalogs"
publicationsByAuthor author1 []
"------------------------------------"
booksByAuthor author1 []
"------------------------------------"
journalsByAuthor author1 []
"------------------------------------"
dvdsByAuthor author2 []

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
,Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"------------------------------------"

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

...

[]

"------------------------------------"

[]

"------------------------------------"

[]

## 7. Functions publicationsByTitle String Catalog and publicationsByDate Date Catalog that return all the publications with that title or release date. 

In [40]:
-- Function to find all publications with the given title
publicationsByTitle :: String -> Catalog -> [Publication]
publicationsByTitle title catalog =
  filter hasTitle catalog
  where
    -- Checks if a publication has the given title
    hasTitle :: Publication -> Bool
    hasTitle (Book _ _ pubTitle _ _) = pubTitle == title
    hasTitle (Journal _ _ pubTitle _ _ _) = pubTitle == title
    hasTitle (DVD _ _ _ pubTitle _ _ _) = pubTitle == title

-- Function to find all publications with the given release date
publicationsByDate :: Date -> Catalog -> [Publication]
publicationsByDate date catalog =
  filter hasReleaseDate catalog
  where
    -- Checks if a publication has the given release date
    hasReleaseDate :: Publication -> Bool
    hasReleaseDate (Book _ _ _ _ _) = False  -- Books don't have a specific release date in this case
    hasReleaseDate (Journal _ _ _ pubReleaseDate _ _) = pubReleaseDate == date
    hasReleaseDate (DVD _ _ _ _ pubReleaseDate _ _) = pubReleaseDate == date



Line 3: Eta reduce
Found:
publicationsByTitle title catalog = filter hasTitle catalog
Why not:
publicationsByTitle title = filter hasTitleLine 14: Eta reduce
Found:
publicationsByDate date catalog = filter hasReleaseDate catalog
Why not:
publicationsByDate date = filter hasReleaseDateLine 19: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}

# Tests for publicationByTitle and publicationsByTitle

In [41]:
let catalog = [book1, journal1, dvd1] -- Replace with your actual catalog of publications

-- Find all publications with the title "Haskell Programming"
let publicationsWithTitle = publicationsByTitle "Pride and Prejudice" catalog
print publicationsWithTitle
"----------------------------------------------------------------------"
-- Find all publications released on a specific date (e.g., "1 Dec 2024")
let date = Date 1 "January" 2024
let publicationsWithDate = publicationsByDate date catalog
print publicationsWithDate
"----------------------------------------------------------------------"
publicationsByDate date []
"----------------------------------------------------------------------"
publicationsByTitle "Pride and Prejudice" []

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"----------------------------------------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"----------------------------------------------------------------------"

[]

"----------------------------------------------------------------------"

[]

## 8. A search Author Title Date function that can receive the author, the title and/or the date and returns all the publications matching the criteria. Notice that the function must work for any combination of the three items (for example searching just by author, or by author and date) and that some, or even the three of them, could be empty.

In [42]:
searchAuthorTitleDate :: Maybe Person -> Maybe String -> Maybe Date -> Catalog -> [Publication]
searchAuthorTitleDate author title date catalog = filter matchesCriteria catalog
  where
    -- Checks if a publication matches the given criteria
    matchesCriteria :: Publication -> Bool
    matchesCriteria pub =
      (authorMatches pub) && (titleMatches pub) && (dateMatches pub)

    -- Check if the publication matches the author criterion
    authorMatches :: Publication -> Bool
    authorMatches (Book _ bookAuthor _ _ _) = maybe True (== bookAuthor) author
    authorMatches (Journal _ journalAuthors _ _ _ _) = maybe True (`elem` journalAuthors) author
    authorMatches (DVD _ director actors _ _ _ _) = maybe True (== director) author || maybe False (`elem` actors) author

    -- Check if the publication matches the title criterion
    titleMatches :: Publication -> Bool
    titleMatches (Book _ _ bookTitle _ _) = maybe True (== bookTitle) title
    titleMatches (Journal _ _ journalTitle _ _ _) = maybe True (== journalTitle) title
    titleMatches (DVD _ _ _ dvdTitle _ _ _) = maybe True (== dvdTitle) title

    -- Check if the publication matches the date criterion
    dateMatches :: Publication -> Bool
    dateMatches (Book _ _ _ _ _) = True  -- Books are not filtered by date in this case
    dateMatches (Journal _ _ _ journalDate _ _) = maybe True (== journalDate) date
    dateMatches (DVD _ _ _ _ dvdDate _ _) = maybe True (== dvdDate) date


Line 2: Eta reduce
Found:
searchAuthorTitleDate author title date catalog
  = filter matchesCriteria catalog
Why not:
searchAuthorTitleDate author title date = filter matchesCriteriaLine 7: Redundant bracket
Found:
(authorMatches pub) && (titleMatches pub) && (dateMatches pub)
Why not:
authorMatches pub && (titleMatches pub) && (dateMatches pub)Line 7: Redundant bracket
Found:
(titleMatches pub) && (dateMatches pub)
Why not:
titleMatches pub && (dateMatches pub)Line 7: Redundant bracket
Found:
(titleMatches pub) && (dateMatches pub)
Why not:
(titleMatches pub) && dateMatches pubLine 23: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}

# Tests for searchAuthorTitleDate

In [43]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

book1 :: Publication
book1 = Book "B001" author1 "Pride and Prejudice" 432 []

book2 :: Publication
book2 = Book "B002" author2 "Oliver Twist" 300 []

journal1 :: Publication
journal1 = Journal "J001" [author1, author2] "Victorian Literature" (Date 1 "January" 2024) 100 []

dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 []

testCatalog :: Catalog
testCatalog = [book1, book2, journal1, dvd1]
let catalog = [book1, journal1, dvd1] -- Replace with actual catalog

-- Search for publications by "John Doe" (author), with title "Haskell Programming"
let results = searchAuthorTitleDate (Just (Person "Jane" "Austen" (Date 16 "December" 1775))) (Just "Pride and Prejudice") Nothing catalog
print results
"----------------------------------------------------"
-- Search for publications with the title "Haskell Programming", regardless of author or date
let resultsByTitle = searchAuthorTitleDate Nothing (Just "Victorian Literature") Nothing catalog
print resultsByTitle
"----------------------------------------------------"
-- Search for publications by author "John Doe" released on a specific date
let resultsByAuthorAndDate = searchAuthorTitleDate (Just (Person "Charles" "Dickens" (Date 7 "February" 1812))) Nothing (Just (Date 1 "April" 2024)) catalog
print resultsByAuthorAndDate
"----------------------------------------------------"
searchAuthorTitleDate Nothing Nothing Nothing catalog
"----------------------------------------------------"
searchAuthorTitleDate Nothing Nothing Nothing []

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
]

"----------------------------------------------------"

[Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
]

"----------------------------------------------------"

[DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/0 available
]

"----------------------------------------------------"

[Book [B001]
  Title: Pride and Prejudice
  Author: Jane Austen (16 December 1775)
  Pages: 432
  Copies: 0/0 available
,Journal [J001]
  Title: Victorian Literature
  Authors: Jane Austen (16 December 1775) Charles Dickens (7 February 1812)
  Release Date: 1 January 2024
  Pages: 100
  Copies: 0/0 available
,DVD [D001]
  Title: Literature on Screen
  Director: Director One (1 January 1980)
  Actors: Actor One (1 January 1990) Charles Dickens (7 February 1812)
  Release Date: 1 April 2024
  Duration: 120 minutes
  Copies: 0/0 available
]

"----------------------------------------------------"

[]

## 9. A nextWorkingDate days Date [Date] function that receives a positive number of days, a Date and a list of holidays and returns the next working day (not in weekend or holiday) after those days. It will be used to calculate loan and return dates for publications, as the library is closed during weekends. Consider leap years and the fact that 1st January 2025 will be Wednesday. 

In [44]:
-- Mapping months and days
months :: [(String, Int)]
months =
  [ ("January", 1), ("February", 2), ("March", 3), ("April", 4),
    ("May", 5), ("June", 6), ("July", 7), ("August", 8),
    ("September", 9), ("October", 10), ("November", 11), ("December", 12) ]

-- days uin months considering leap years
daysInMonth :: String -> Int -> Int
daysInMonth "February" year = if isLeapYear year then 29 else 28
daysInMonth month _
  | month `elem` ["April", "June", "September", "November"] = 30
  | otherwise = 31

isLeapYear :: Int -> Bool
isLeapYear year
  | year `mod` 400 == 0 = True
  | year `mod` 100 == 0 = False
  | year `mod` 4 == 0 = True
  | otherwise = False

-- Auxiliar function to add days
addDays :: Int -> Date -> Date
addDays 0 date = date
addDays n (Date d m y)
  | n + d <= daysInMonth m y = Date (d + n) m y -- Reuslting day in the actual month
  | otherwise =
      let daysLeft = n - (daysInMonth m y - d) -- Whenever you fill the actual month, compute days left
          (nextMonth, nextYear) = nextMonthYear m y -- Next month or year if necessary
      in addDays (daysLeft - 1) (Date 1 nextMonth nextYear) -- Get back to day 1 and continue


nextMonthYear :: String -> Int -> (String, Int)
nextMonthYear "December" year = ("January", year + 1)
nextMonthYear month year =
  case lookup month months of
    Just num -> case lookup (num + 1) (map (\(m, n) -> (n, m)) months) of
                  Just next -> (next, year)
                  Nothing -> error "Invalid month"
    Nothing -> error "Invalid month"

-- CDays between dates
daysFromStartOfYear :: Date -> Int
daysFromStartOfYear (Date d m y) =
  let monthDays = sum [daysInMonth month y | (month, _) <- takeWhile ((/= m) . fst) months]
  in monthDays + d

daysUntilEndOfYear :: Date -> Int
daysUntilEndOfYear (Date d m y) =
  let monthDays = sum [daysInMonth month y | (month, _) <- dropWhile ((/= m) . fst) months]
  in monthDays - d

daysInYear :: Int -> Int
daysInYear year = if isLeapYear year then 366 else 365

daysBetween :: Date -> Date -> Int
daysBetween date1@(Date d1 m1 y1) date2@(Date d2 m2 y2)
  | y1 == y2 = daysFromStartOfYear date2 - daysFromStartOfYear date1
  | y1 < y2  = let daysFirstYear = daysUntilEndOfYear date1
                   daysLastYear = daysFromStartOfYear date2
                   daysInMiddleYears = sum [daysInYear year | year <- [y1 + 1 .. y2 - 1]]
               in daysFirstYear + daysInMiddleYears + daysLastYear
  | otherwise = let daysFirstYear = daysUntilEndOfYear date2
                    daysLastYear = daysFromStartOfYear date1
                    daysInMiddleYears = sum [daysInYear year | year <- [y2 + 1 .. y1 - 1]]
                in daysFirstYear + daysInMiddleYears + daysLastYear

-- DWeekday
daysOfWeek :: [String]
daysOfWeek = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

dayOfWeek :: Date -> String
dayOfWeek (Date d m y) =
  let refDate = Date 1 "January" 2025 -- Miércoles
      daysDiff = daysBetween refDate (Date d m y)
  in daysOfWeek !! ((2 + daysDiff) `mod` 7)

-- Check for working day
isWorkingDay :: Date -> [Date] -> Bool
isWorkingDay date holidays =
  let weekday = dayOfWeek date
  in weekday `notElem` ["Saturday", "Sunday"] && date `notElem` holidays

-- Next working date function
nextWorkingDate :: Int -> Date -> [Date] -> Date
nextWorkingDate days startDate holidays =
  let targetDate = addDays days startDate
  in if isWorkingDay targetDate holidays
       then targetDate
       else nextWorkingDate 1 targetDate holidays

# Tests for nextWorkingDate and auxiliar functions

In [45]:
-- test with the first date being holidays
let startDate = Date 1 "January" 2025
    holidays = [Date 1 "January" 2025,Date 6 "January" 2025] -- Ejemplo: Día de Reyes
nextWorkingDate 1 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2025)

2 January 2025

13 January 2025

"Sunday"

In [46]:
-- test change of year
let startDate = Date 30 "December" 2025
    holidays = [Date 1 "January" 2026,Date 6 "January" 2026] -- Ejemplo: Día de Reyes
--addDays 2 startDate
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2026)
"------------"
nextMonthYear "December" 2025
--2 - (daysInMonth "December" 2025 - 31)

2 January 2026

9 January 2026

"Monday"

"------------"

("January",2026)

In [47]:
--test days of week 
let startDate = Date 31 "December" 2026
    holidays = [Date 1 "January" 2027,Date 6 "January" 2027] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2027)
dayOfWeek (Date 27 "February" 2028)
dayOfWeek (Date 28 "February" 2028)
dayOfWeek (Date 29 "February" 2028)
dayOfWeek (Date 1 "March" 2028)

4 January 2027

11 January 2027

"Tuesday"

"Sunday"

"Monday"

"Tuesday"

"Wednesday"

In [48]:
let startDate = Date 31 "December" 2025
    holidays = [Date 1 "January" 2026,Date 6 "January" 2026] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays

2 January 2026

12 January 2026

In [ ]:
let startDate = Date 31 "December" 2026
    holidays = [Date 1 "January" 2027,Date 6 "January" 2027] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays

4 January 2027

11 January 2027

In [ ]:
let startDate = Date 31 "December" 2025
    holidays = [Date 1 "January" 2026,Date 6 "January" 2026] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2026)
dayOfWeek (Date 12 "January" 2026)

2 January 2026

12 January 2026

"Monday"

"Monday"

In [ ]:
let startDate = Date 31 "December" 2026
    holidays = [Date 1 "January" 2027,Date 6 "January" 2027] -- Ejemplo: Día de Reyes
nextWorkingDate 2 startDate holidays
nextWorkingDate 10 startDate holidays
dayOfWeek (Date 5 "January" 2027)
dayOfWeek (Date 27 "February" 2028)
dayOfWeek (Date 28 "February" 2028)
dayOfWeek (Date 29 "February" 2028)
dayOfWeek (Date 1 "March" 2028)

4 January 2027

11 January 2027

"Tuesday"

"Sunday"

"Monday"

"Tuesday"

"Wednesday"

In [ ]:
let startDate = Date 31 "December" 2025
    holidays = [Date 1 "January" 2026, Date 6 "January" 2026] -- Ejemplo: Día de Reyes

-- Prueba de addDays
addDays 2 startDate


2 January 2026

### 10. A borrow User Publication Date Catalog function that borrows the publication starting in that date. If no copy is available on that date, the publication will be borrowed at the earliest possible date. It will return the updated Catalog. 

In [ ]:
-- defining instances that we will be using later in the function
instance Eq Date where
  (Date d1 m1 y1) == (Date d2 m2 y2) =
    y1 == y2 && monthIndex m1 == monthIndex m2 && d1 == d2

instance Ord Date where
  compare (Date d1 m1 y1) (Date d2 m2 y2) =
    compare (y1, monthIndex m1, d1) (y2, monthIndex m2, d2)
    
holidays = [Date 15 "January" 2026, Date 16 "January" 2026]
    -- Convert month name to an index
monthIndex :: String -> Int
monthIndex month = case month of
    "January" -> 1; "February" -> 2; "March" -> 3; "April" -> 4; "May" -> 5
    "June" -> 6; "July" -> 7; "August" -> 8; "September" -> 9; "October" -> 10
    "November" -> 11; "December" -> 12; _ -> error "Invalid month"

In [ ]:
findIndex :: (a -> Bool) -> [a] -> Maybe Int
findIndex p xs = findIndex' p xs 0
  where
    findIndex' _ [] _ = Nothing
    findIndex' p (x:xs) i
      | p x = Just i
      | otherwise = findIndex' p xs (i + 1)
      
maybeToList :: Maybe a -> [a]
maybeToList Nothing = []
maybeToList (Just x) = [x]

-- Based on type of pub and user, duration of the loan
loanDuration :: User -> Publication -> Int
loanDuration (Student _) (Book _ _ _ _ _) = 30      -- 1 mes
loanDuration (Student _) (Journal _ _ _ _ _ _) = 15   -- 15 días
loanDuration (Student _) (DVD _ _ _ _ _ _ _) = 7     -- 1 semana
loanDuration (Professor _) (Book _ _ _ _ _) = 60   -- 2 meses
loanDuration (Professor _) (Journal _ _ _ _ _ _) = 30 -- 1 mes
loanDuration (Professor _) (DVD _ _ _ _ _ _ _) = 15  -- 15 días

findAvailableCopy :: Publication -> Date -> Maybe Int
findAvailableCopy publication currentDate =
  let copies = getCopies publication
  in findIndex (\copy -> not (isBorrowed copy) || isOverdue copy currentDate) copies


-- DCheck if a copy is overdue and thus available 
isOverdue :: Copy -> Date -> Bool
isOverdue copy currentDate =
  isBorrowed copy && case returnDate copy of
    Just rd -> rd < currentDate -- Returning date happened 
    Nothing -> False -- No returning date, then not available 



-- Get copies of a piblication
getCopies :: Publication -> [Copy]
getCopies (Book _ _ _ _ copies) = copies
getCopies (Journal _ _ _ _ _ copies) = copies
getCopies (DVD _ _ _ _ _ _ copies) = copies

-- Update copy with info of the user
updateCopy :: Copy -> User -> Date -> Date -> Copy
updateCopy copy user loanDate returnDate =
  copy { isBorrowed = True, borrower = Just user, loanDate = Just loanDate, returnDate = Just returnDate }

-- Update copy list from a pub
updateCopies :: [Copy] -> Int -> Copy -> [Copy]
updateCopies copies index newCopy = take index copies ++ [newCopy] ++ drop (index + 1) copies

-- Update a publication with new copies
updatePublication :: Publication -> [Copy] -> Publication
updatePublication (Book id author title pages _) copies = Book id author title pages copies
updatePublication (Journal id authors title releaseDate pages _) copies = Journal id authors title releaseDate pages copies
updatePublication (DVD id director actors title releaseDate duration _) copies = DVD id director actors title releaseDate duration copies

-- Replace a publication from the catalog
replacePublication :: [Publication] -> Publication -> Publication -> [Publication]
replacePublication [] _ _ = []
replacePublication (x:xs) old new
  | x == old = new : xs
  | otherwise = x : replacePublication xs old new

-- Próxima fecha disponible
--nextAvailableDate :: Publication -> Date -> [Publication] -> Date
--nextAvailableDate publication date catalog =
  --let unavailableDates = concatMap (\copy -> maybeToList (returnDate copy)) (getCopies publication)
  --in head [nextDate | nextDate <- iterate (addDays 1) date, nextDate `notElem` unavailableDates]
nextAvailableDate :: Publication -> Date -> Date
nextAvailableDate publication date =
  let copies = getCopies publication
      returnDates = concatMap (\copy -> maybeToList (returnDate copy)) copies
  in case returnDates of
       [] -> error "No return dates available for this publication" -- Manejo de error explícito
       _  -> let earliestReturn = minimum returnDates -- La primera fecha en la que se devuelve una copia
                 adjustedDate = if date > earliestReturn then date else earliestReturn
             in adjustedDate

borrow :: User -> Publication -> Date -> [Publication] -> [Publication]
borrow user publication startDate catalog =
  let updatedCatalog = returnOverdueCopies user startDate catalog -- Paso 1: Give back copies overdue
  in if length (borrowedPublications user updatedCatalog) >= maxBorrowed user
        then updatedCatalog -- Paso 2: If it exceed the limit, do not continue with the loan
        else -- Paso 3: Continue with the loan
          let availableCopyIndex = findAvailableCopy publication (nextWorkingDate 0 startDate holidays)
          in case availableCopyIndex of
               Just index -> -- Copy available
                 let loanPeriod = loanDuration user publication
                     returnDate = nextWorkingDate 0 (addDays loanPeriod (nextWorkingDate 0 startDate holidays)) holidays
                     updatedCopy = updateCopy (getCopies publication !! index) user (nextWorkingDate 0 startDate holidays) returnDate
                     updatedCopies = updateCopies (getCopies publication) index updatedCopy
                     updatedPublication = updatePublication publication updatedCopies
                 in replacePublication updatedCatalog publication updatedPublication
               Nothing -> -- NO copies avaiblable look for the next date 
                 borrow user publication (nextWorkingDate 1 (nextWorkingDate 0 startDate holidays) holidays) updatedCatalog

-- Give back automatically copies overdue
returnOverdueCopies :: User -> Date -> [Publication] -> [Publication]
returnOverdueCopies user currentDate catalog = map (returnOverdueCopiesFromPublication user currentDate) catalog

-- Check and give back copies in a pub
returnOverdueCopiesFromPublication :: User -> Date -> Publication -> Publication
returnOverdueCopiesFromPublication user currentDate publication =
  let updatedCopies = map (returnIfOverdue user currentDate) (getCopies publication)
  in updatePublication publication updatedCopies

-- If an overdue copy belongs to user, then return it
returnIfOverdue :: User -> Date -> Copy -> Copy
returnIfOverdue user currentDate copy
  | isBorrowed copy && borrower copy == Just user && isOverdue copy currentDate =
      copy { isBorrowed = False, borrower = Nothing, loanDate = Nothing, returnDate = Nothing }
  | otherwise = copy


-- Determine the maximum number of publications a user can borrow
maxBorrowed :: User -> Int
maxBorrowed (Student _) = 3
maxBorrowed (Professor _) = 5




Line 15: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 16: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 17: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 18: Use record patterns
Found:
Book _ _ _ _ _
Why not:
Book {}Line 19: Use record patterns
Found:
Journal _ _ _ _ _ _
Why not:
Journal {}Line 20: Use record patterns
Found:
DVD _ _ _ _ _ _ _
Why not:
DVD {}Line 73: Avoid lambda
Found:
\ copy -> maybeToList (returnDate copy)
Why not:
maybeToList . returnDateLine 77: Use max
Found:
if date > earliestReturn then date else earliestReturn
Why not:
max date earliestReturnLine 100: Eta reduce
Found:
returnOverdueCopies user currentDate catalog
  = map (returnOverdueCopiesFromPublication user currentDate) catalog
Why not:
returnOverdueCopies user currentDate
  = map (returnOverdueCopiesFromPublication user currentDate)

# Test for borrow

In [ ]:
-- Usuarios
let user1 = Student (Person "Alice" "Smith" (Date 1 "January" 2000))
let user2 = Professor (Person "Bob" "Johnson" (Date 15 "March" 1980))

-- Publicación con copia vencida
let book2 = Book "B2" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [ Copy True (Just user2) (Just (Date 28 "November" 2025)) (Just (Date 30 "January" 2026)) ]

-- Fecha actual: después del vencimiento
let currentDate = Date 2 "January" 2026
findAvailableCopy book2 currentDate
findAvailableCopy book2 (Date 31 "January" 2026)
--isOverdue (Copy True (Just user2) (Just (Date 28 "November" 2025)) (Just (Date 28 "December" 2025))) currentDate
--Date 2 "January" 2026 > (Date 28 "December" 2025)
-- Intento de préstamo
let result = borrow user1 book2 currentDate [book2]


Nothing

Just 0

In [ ]:
result

[Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 2 February 2026 | Return Date: 4 March 2026
]

In [ ]:
--Pruebas
-- Usuarios
holidays = [Date 15 "January" 2026, Date 16 "January" 2026]
let user1 = Student (Person "Alice" "Smith" (Date 1 "January" 2000))
let user2 = Professor (Person "Bob" "Johnson" (Date 15 "March" 1980))

-- Publicaciones y copias
let book1 = Book "B1" (Person "Author" "One" (Date 1 "January" 1970)) "Functional Programming" 200
                  [ Copy True (Just user1) (Just (Date 1 "December" 2025)) (Just (Date 31 "December" 2025))
                  , Copy False Nothing Nothing Nothing ]

let book2 = Book "B2" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [ Copy True (Just user2) (Just (Date 28 "November" 2025)) (Just (Date 15 "December" 2025)) ]

-- Catálogo
let catalog = [book1, book2]


In [ ]:
catalog
"---------------------"
borrow user2 book1 (Date 2 "December" 2025) catalog
"---------------------"
borrow user1 book2 (Date 2 "December" 2025) catalog

[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 1/2 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 1 December 2025 | Return Date: 31 December 2025
Available
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Professor: Bob Johnson (15 March 1980)
 | Loan Date: 28 November 2025 | Return Date: 15 December 2025
]

"---------------------"

[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 0/2 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 1 December 2025 | Return Date: 31 December 2025
Borrowed by Professor: Bob Johnson (15 March 1980)
 | Loan Date: 2 December 2025 | Return Date: 2 February 2026
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Professor: Bob Johnson (15 March 1980)
 | Loan Date: 28 November 2025 | Return Date: 15 December 2025
]

"---------------------"

[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 1/2 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 1 December 2025 | Return Date: 31 December 2025
Available
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 0/1 available
Borrowed by Student: Alice Smith (1 January 2000)
 | Loan Date: 16 December 2025 | Return Date: 19 January 2026
]

In [ ]:
--prueba top
-- Users
let user1 = Student (Person "Alice" "Smith" (Date 1 "January" 2000))
let user2 = Professor (Person "Bob" "Johnson" (Date 15 "March" 1980))
let user3 = Student (Person "Charlie" "Brown" (Date 1 "January" 2000))
-- Publications
let book1 = Book "B1" (Person "Author" "One" (Date 1 "January" 1970)) "Functional Programming" 200
                  [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]

let book2 = Book "B2" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [Copy False Nothing Nothing Nothing, Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]
let book3 = Book "B3" (Person "Author" "Two" (Date 2 "February" 1980)) "Advanced Haskell" 300
                  [Copy False Nothing Nothing Nothing,  Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024))]
-- Catalog
let catalog = [book1, book2, book3]
borrow user3 book2 (Date 2 "December" 2024) catalog
-- Borrow Attempt
{-let result1 = borrow user1 book1 (Date 2 "December" 2024) catalog
let result2 = borrow user2 book1 (Date 2 "December" 2024) result1
let user3 = Student (Person "Charlie" "Brown" (Date 1 "January" 2000))
let catalogWithLoans = [ Book "B3" (Person "Author" "Three" (Date 3 "March" 1990)) "Learn Haskell" 250
                           [ Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                             Copy True (Just user3) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                             Copy False Nothing Nothing Nothing ]]
let result3 = borrow user3 book2 (Date 2 "December" 2024) catalogWithLoans
let user4 = Student (Person "Dana" "Scott" (Date 1 "January" 2000))
let catalogWithMaxLoans = [ Book "B4" (Person "Author" "Four" (Date 4 "April" 2000)) "Functional Magic" 150
                             [ Copy True (Just user4) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                               Copy True (Just user4) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)),
                               Copy True (Just user4) (Just (Date 1 "December" 2024)) (Just (Date 31 "December" 2024)) ]]
let result4 = borrow user4 book2 (Date 2 "December" 2024) catalogWithMaxLoans


[Book [B1]
  Title: Functional Programming
  Author: Author One (1 January 1970)
  Pages: 200
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
,Book [B2]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
,Book [B3]
  Title: Advanced Haskell
  Author: Author Two (2 February 1980)
  Pages: 300
  Copies: 1/2 available
Available
Borrowed by Student: Charlie Brown (1 January 2000)
 | Loan Date: 1 December 2024 | Return Date: 31 December 2024
]

### 11. Functions to use I/O to add, remove, search and borrow publications, as well as function to show the catalog or the information about a user.

In [ ]:
import Data.Maybe (fromMaybe)
processFile :: FilePath -> IO (Either String (Either Catalog [Publication]))
processFile filePath = do
    content <- readFile filePath
    let allLines = lines content
    case allLines of
        [] -> return $ Left "Error: The file is empty."
        (op : rest) -> do
            let operation = read op :: Int
            case operation of
                1 -> do
                    let sections = splitOnSeparator "===" rest
                    case sections of
                        [publicationSection, catalogSection] -> do
                            let publication = parsePublication publicationSection
                            let fileCatalog = parseCatalog catalogSection
                            return $ Right $ Left $ addToCatalog publication fileCatalog
                        _ -> return $ Left "Error: Invalid format for operation 'Add'."
                2 -> do
                    let sections = splitOnSeparator "===" rest
                    case sections of
                        [publicationSection, catalogSection] -> do
                            let publication = parsePublication publicationSection
                            let fileCatalog = parseCatalog catalogSection
                            case removeFromCatalog publication fileCatalog of
                                Left err -> return $ Left err
                                Right updatedCatalog -> return $ Right $ Left updatedCatalog
                        _ -> return $ Left "Error: Invalid format for operation 'Remove'."
                3 -> do
                    let sections = splitOnSeparator "===" rest
                    case sections of
                        [criteriaSection, catalogSection] -> do
                            let (author, title, date) = parseSearchCriteria criteriaSection
                            let fileCatalog = parseCatalog catalogSection
                            let results = searchAuthorTitleDate author title date fileCatalog
                            return $ Right $ Right results
                        _ -> return $ Left "Error: Invalid format for operation 'Search'."
                4 -> do
                    let sections = splitOnSeparator "===" rest
                    case sections of
                        [userSection, publicationSection, dateSection, catalogSection] -> do
                            let user = parseUser (head userSection)
                            let publication = parsePublication publicationSection
                            let startDate = parseDate (head dateSection)
                            let fileCatalog = parseCatalog catalogSection
                            let updatedCatalog = borrow user publication startDate fileCatalog
                            return $ Right $ Left updatedCatalog
                        _ -> return $ Left "Error: Invalid format for operation 'Borrow'."
                5 -> do
                    let catalogSection = rest
                    let fileCatalog = parseCatalog catalogSection
                    prettyCatalog fileCatalog
                    return $ Left "Result printed correctly"
                6 -> do
                    let userLine = head rest
                    let user = parseUser userLine
                    return $ Left $ show user
                _ -> return $ Left "Error: Unsupported operation."
-- Parse string to create a publication
parsePublication :: [String] -> Publication
parsePublication (typeTag : args) = case typeTag of
    "Book"    -> parseBook args
    "Journal" -> parseJournal args
    "DVD"     -> parseDVD args
    _         -> error "Tipo de publicación no reconocido"
parsePublication _ = error "Datos insuficientes para parsear publicación"

-- Parse a Book
parseBook :: [String] -> Publication
parseBook [bookId, authorFirst, authorLast, dayStr, monthStr, yearStr, title, pagesStr, copiesStr] =
    Book bookId author title (read pagesStr) (parseCopies copiesStr)
  where
    author = Person authorFirst authorLast (Date (read dayStr) monthStr (read yearStr))
parseBook _ = error "Formato inválido para Book"

-- Parse a Journal
parseJournal :: [String] -> Publication
parseJournal (journalId : authorsStr : dayStr : monthStr : yearStr : title : pagesStr : copiesStr : []) =
    Journal journalId (parseAuthors authorsStr) title releaseDate (read pagesStr) (parseCopies copiesStr)
  where
    releaseDate = Date (read dayStr) monthStr (read yearStr)
parseJournal _ = error "Formato inválido para Journal"

-- Parse a DVD
parseDVD :: [String] -> Publication
parseDVD (dvdId : directorFirst : directorLast : actorsStr : dayStr : monthStr : yearStr : title : durationStr : copiesStr : []) =
    DVD dvdId director actors title releaseDate (read durationStr) (parseCopies copiesStr)
  where
    director = Person directorFirst directorLast (Date (read dayStr) monthStr (read yearStr))
    actors = parseAuthors actorsStr
    releaseDate = Date (read dayStr) monthStr (read yearStr)
parseDVD _ = error "Formato inválido para DVD"

-- Parse from a string a list of authors
parseAuthors :: String -> [Person]
parseAuthors authorsStr = map parseAuthor (split ',' authorsStr)

-- Parse an author (Person)
parseAuthor :: String -> Person
parseAuthor str = case split '/' str of
    [first, last, dayStr, monthStr, yearStr] ->
        Person first last (Date (read dayStr) monthStr (read yearStr))
    _ -> error "Formato inválido para autor"

-- Parse copy list from a string
parseCopies :: String -> [Copy]
parseCopies copiesStr = map parseCopy (split ',' copiesStr)

-- Parse a copy
parseCopy :: String -> Copy
parseCopy str = case split '|' str of
    [isBorrowedStr, borrowerStr, loanDayStr, loanMonthStr, loanYearStr, returnDayStr, returnMonthStr, returnYearStr] ->
        Copy
            (read isBorrowedStr)
            (parseMaybeUser borrowerStr)
            (parseMaybeDate loanDayStr loanMonthStr loanYearStr)
            (parseMaybeDate returnDayStr returnMonthStr returnYearStr)
    _ -> error "Formato inválido para copia"
    
parseMaybeUser :: String -> Maybe User
parseMaybeUser "-" = Nothing
parseMaybeUser userStr = Just $ case split '/' userStr of
    ["Student", first, last, dayStr, monthStr, yearStr] ->
        Student $ Person first last (Date (read dayStr) monthStr (read yearStr))
    ["Professor", first, last, dayStr, monthStr, yearStr] ->
        Professor $ Person first last (Date (read dayStr) monthStr (read yearStr))
    _ -> error $ "Formato inválido para usuario: " ++ userStr

-- Parse a Maybe Date
parseMaybeDate :: String -> String -> String -> Maybe Date
parseMaybeDate "-" "-" "-" = Nothing
parseMaybeDate dayStr monthStr yearStr =
    Just $ Date (read dayStr) monthStr (read yearStr)

-- Auxiliar function to split strings
split :: Char -> String -> [String]
split delim str = case break (== delim) str of
    (a, _ : rest) -> a : split delim rest
    (a, "")       -> [a]

--Add Remove Search
parseSearchCriteria :: [String] -> (Maybe Person, Maybe String, Maybe Date)
parseSearchCriteria [] = (Nothing, Nothing, Nothing)
parseSearchCriteria (authorStr : titleStr : dateStr : []) =
    (parseMaybePerson authorStr, parseMaybeString titleStr, parseMaybeDateString dateStr)
parseSearchCriteria _ = error "Error: Invalid search criteria format."

parseMaybePerson :: String -> Maybe Person
parseMaybePerson "-" = Nothing
parseMaybePerson str = case split '/' str of
    [first, last, dayStr, monthStr, yearStr] ->
        Just $ Person first last (Date (read dayStr) monthStr (read yearStr))
    _ -> error "Error: Invalid person format."

parseMaybeString :: String -> Maybe String
parseMaybeString "-" = Nothing
parseMaybeString str = Just str

parseMaybeDateString :: String -> Maybe Date
parseMaybeDateString "-" = Nothing
parseMaybeDateString str = case split '/' str of
    [dayStr, monthStr, yearStr] ->
        Just $ Date (read dayStr) monthStr (read yearStr)
    _ -> error "Error: Invalid date format."

parseDate :: String -> Date
parseDate str = case split '/' str of
    [dayStr, monthStr, yearStr] ->
        Date (read dayStr) monthStr (read yearStr)
    _ -> error $ "Error: Invalid date format: " ++ str

parseBorrowInput :: [String] -> (User, Publication, Date, Catalog)
parseBorrowInput lines =
    let sections = splitOnSeparator "===" lines
    in case sections of
        [userSection, publicationSection, dateSection, catalogSection] ->
            ( parseUser (head userSection)
            , parsePublication publicationSection
            , parseDate (head dateSection)
            , parseCatalog catalogSection
            )
        _ -> error "Error: Invalid input format. Expected 4 sections separated by '==='."

-- Helper: Parse catalog with * as separator
parseCatalog :: [String] -> Catalog
parseCatalog = map parsePublication . splitOnSeparator "*"


parseUser :: String -> User
parseUser str = case split '/' str of
    ["Student", first, last, dayStr, monthStr, yearStr] ->
        Student $ Person first last (Date (read dayStr) monthStr (read yearStr))
    ["Professor", first, last, dayStr, monthStr, yearStr] ->
        Professor $ Person first last (Date (read dayStr) monthStr (read yearStr))
    _ -> error "Error: Invalid user format."

-- Utility to split lines into chunks (separated by blank lines)
splitOnEmpty :: [String] -> [[String]]
splitOnEmpty = filter (not . null) . foldr splitHelper [[]]
  where
    splitHelper "" acc = [] : acc
    splitHelper x (y:ys) = (x:y) : ys
    
-- Helper: Split input into sections based on the separator
splitOnSeparator :: String -> [String] -> [[String]]
splitOnSeparator sep = filter (not . null) . foldr splitHelper [[]]
  where
    splitHelper line acc
        | line == sep = [] : acc
        | otherwise = (line : head acc) : tail acc
        
-- Break the list at a separator line
breakOn :: String -> [String] -> ([String], String, [String])
breakOn sep xs = let (first, rest) = break (== sep) xs
                 in case rest of
                      [] -> error "Error: Separator not found."
                      (_ : remaining) -> (first, sep, remaining)




Line 78: Use list literal pattern
Found:
(journalId : authorsStr : dayStr : monthStr : yearStr : title : pagesStr : copiesStr : [])
Why not:
[journalId,
 authorsStr,
 dayStr,
 monthStr,
 yearStr,
 title,
 pagesStr,
 copiesStr]Line 86: Use list literal pattern
Found:
(dvdId : directorFirst : directorLast : actorsStr : dayStr : monthStr : yearStr : title : durationStr : copiesStr : [])
Why not:
[dvdId,
 directorFirst,
 directorLast,
 actorsStr,
 dayStr,
 monthStr,
 yearStr,
 title,
 durationStr,
 copiesStr]Line 144: Use list literal pattern
Found:
(authorStr : titleStr : dateStr : [])
Why not:
[authorStr, titleStr, dateStr]

In [ ]:
let catalog = []  -- Catalogo inicial vacío
updatedCatalog <- processFile "ExtraAdd.txt"
print updatedCatalog


Right (Left [Book [B002]
  Title: Another Book
  Author: Jane Doe (1 January 2000)
  Pages: 150
  Copies: 1/1 available
Available
,Journal [J001]
  Title: Functional Programming Advances
  Authors: Author1 Smith (20 April 1975) Author2 Jones (5 March 1980)
  Release Date: 10 December 2020
  Pages: 120
  Copies: 1/2 available
Borrowed by Professor: Mary Johnson (15 May 1960)
 | Loan Date: 5 December 2020 | Return Date: 10 December 2020
Available
,Book [B003]
  Title: Tiana y el Sapo
  Author: Janey Doeg (4 January 2004)
  Pages: 250
  Copies: 1/1 available
Available
])

In [ ]:
author1 :: Person
author1 = Person "Jane" "Austen" (Date 16 "December" 1775)

author2 :: Person
author2 = Person "Charles" "Dickens" (Date 7 "February" 1812)

actor1 :: Person
actor1 = Person "Actor" "One" (Date 1 "January" 1990)

director1 :: Person
director1 = Person "Director" "One" (Date 1 "January" 1980)

book1 :: Publication
book1 = Book "B001" author1 "Pride and Prejudice" 432 []

book2 :: Publication
book2 = Book "B002" author2 "Oliver Twist" 300 []

journal1 :: Publication
journal1 = Journal "J001" [author1, author2] "Victorian Literature" (Date 1 "January" 2024) 100 []

dvd1 :: Publication
dvd1 = DVD "D001" director1 [actor1, author2] "Literature on Screen" (Date 1 "April" 2024) 120 []

catalogFromFile :: Catalog
catalogFromFile = [book1, book2, journal1, dvd1]
--updatedCatalog <- processFile "publications.txt" catalogFromFile
--print updatedCatalog

In [ ]:
--let author1 = Person "Jane" "Doe" (Date 15 "July" 1990)
--let book1 = Book "B001" author1 "Haskell for Beginners" 300 [Copy False Nothing Nothing Nothing]
--let myCatalog = [book1, book2]
--myCatalog
"-------------Add a publication in the catalog----------------"
processFile "ExtraAdd.txt"
"-------------Show info about the user ------------------"
processFile "ExtraShowUser.txt" 
"-------------Show info about the catalog ------------------"
processFile "ExtraShowCatalog.txt" 
"-------------Search ------------------"
processFile "ExtraSearch.txt" 
"-------------Remove------------------"
processFile "ExtraRemove.txt" 
"-------------Borrow------------------"
processFile "ejemploBorrow.txt" 

"-------------Add a publication in the catalog----------------"

Right (Left [Book [B002]
  Title: Another Book
  Author: Jane Doe (1 January 2000)
  Pages: 150
  Copies: 1/1 available
Available
,Journal [J001]
  Title: Functional Programming Advances
  Authors: Author1 Smith (20 April 1975) Author2 Jones (5 March 1980)
  Release Date: 10 December 2020
  Pages: 120
  Copies: 1/2 available
Borrowed by Professor: Mary Johnson (15 May 1960)
 | Loan Date: 5 December 2020 | Return Date: 10 December 2020
Available
,Book [B003]
  Title: Tiana y el Sapo
  Author: Janey Doeg (4 January 2004)
  Pages: 250
  Copies: 1/1 available
Available
])

"-------------Show info about the user ------------------"

Left "Professor: Mary Johnson (15 May 1960)\n"

"-------------Show info about the catalog ------------------"

...

Right (Right [Book [B001]
  Title: Haskell for Beginners
  Author: Jane Doe (15 July 1990)
  Pages: 300
  Copies: 1/1 available
Available
])

"-------------Remove------------------"

Right (Left [Journal [J001]
  Title: Functional Programming Advances
  Authors: Author1 Smith (20 April 1975) Author2 Jones (5 March 1980)
  Release Date: 10 December 2020
  Pages: 120
  Copies: 1/2 available
Borrowed by Professor: Mary Johnson (15 May 1960)
 | Loan Date: 5 December 2020 | Return Date: 10 December 2020
Available
])

"-------------Borrow------------------"

Right (Left [Book [B002]
  Title: Another Book
  Author: Jane Doe (1 January 2000)
  Pages: 150
  Copies: 0/1 available
Borrowed by Student: John Doe (30 August 2003)
 | Loan Date: 1 December 2023 | Return Date: 1 January 2024
,Journal [J001]
  Title: Functional Programming Advances
  Authors: Author1 Smith (20 April 1975) Author2 Jones (5 March 1980)
  Release Date: 10 December 2020
  Pages: 120
  Copies: 1/2 available
Borrowed by Professor: Mary Johnson (15 May 1960)
 | Loan Date: 5 December 2020 | Return Date: 10 December 2020
Available
])

# Structure of the text to use ADD

# Structure of the text to use REMOVE

# Structure of the text to use SEARCH

# Structure of the text to use BORROW

# Structure of the text to use SHOW CATALOG

# Structure of the text to use SHOW USER

# Conclusions

The work performed was complete, we not only implemented the required functions but also tests to ensure their correct performance. 

The 3 main problems we encountered doing the project were:
    
    * The definition of the types was a tricky part as we defined our types in the first place and there were some functions that made us change the original definition of the types and thus change it everywhere in the document. 
    
    * The nextWorkingDate function took us a lot of time as handling the real weekends and vacations to count the working dates was difficult. After a lot of tries, and auxiliary functions we got a perfect-working version of the function. 

    * Handling the IO objects: for exercise number 11 we had a lot of problem as Haskell interprets differently things taken from IO than internal things, as a result of this, handling the input text to get the arguments for the functions that we implemented was not easy at all. We needed to take care of what was the best option, the input text should be redacted and then create a lot of auxiliary functions to help us process the inputed data

As personal comments, we enjoyed ourselves doing this project as it made us really understand what functional programming really means. It was a hard-working project that led to satisfactory results that caused us to be very happy when after trying for so long for a function (ex 11 was big deal) it finally came with the correct output! We wanted to thank also Angel for the help and patience explaining some key parts of the project!